# Capsule CRM - API Test

## Requirements

First of all I strongly suggest you to create a **virtual environment**:

    virtualenv env
    source env/bin/activate

Install the requirements (Be aware to have the virtualenv activated):
    
    pip install -r requirements
    
To have authenticated access to the API you need to create a custom TOKEN. Of course the production TOKEN must be secret!
You can create a personal TOKEN in your **Preferences** under the voice **API Authentication Tokens**.

You need to copy the file **secret.example** with a new name **secret**. Change inside this file the voice **TOKEN** with your token.

## Attention

We can perform only **4,000 requests per hour**. Inside the Header we'll receive an update on the Limit:

    HTTP/1.1 200 OK
    X-RateLimit-Limit: 4000
    X-RateLimit-Remaining: 56
    X-RateLimit-Reset: 1434037662

If we reach the limiti we'll receive something like:

    HTTP/1.1 429 Too Many Requests
    X-RateLimit-Limit: 4000
    X-RateLimit-Remaining: 0
    X-RateLimit-Reset: 1434037662
    
With a Body Response:

    { "error":"rate limit reached" }

## TOKEN Setup

In [ ]:
from configparser import ConfigParser
import os, requests

In [ ]:
parser = ConfigParser()
parser.read(os.path.expanduser('./secret'))
token = parser.get('AUTH', 'token')

In [ ]:
headers = {
    'Authorization': 'Bearer {}'.format(token),
    'Content-Type': 'application/json',
    'Accept': 'application/json'
    }
print headers

If we don't set properly the authorization header we'll receive something like:

    HTTP/1.1 401 Unauthorized
    WWW-Authenticate: Bearer realm="capsule", error="invalid_token"
    {
      "message": "Requires authentication"
    }

## HTTP Method

Call the Wrapper method **perform_get** if you need to perform **GET** calls, so you don't need to set the authorization headers manually each time.

In [ ]:
def perform_get(url):
    """
    Perform HTTP GET with Authorization Headers
    :url: Specify the destination URL
    :return: Requests Response Object
    """
    return requests.get(url=url, headers=headers)

Call the Wrapper method **perform_POST** if you need to perform **POST** calls, so you don't need to set the authorization headers manually each time. You need to pass the body of the request.

In [ ]:
def perform_post(url, data=None):
    """
    Perform HTTP POST with Authorization Headers
    :url: Specify the destination URL
    :data: Content Body
    :return: Requests Response Object
    """
    return requests.post(url=url, data=data, headers=headers)

Call the Wrapper method **perform_PUT** if you need to perform **PUT** calls, so you don't need to set the authorization headers manually each time. You need to pass the body of the request.

In [ ]:
def perform_put(url, data=None):
    """
    Perform HTTP PUT with Authorization Headers
    :url: Specify the destination URL
    :data: Content Body
    :return: Requests Response Object
    """
    return requests.put(url=url, data=data, headers=headers)

Call the Wrapper method **perform_delete** if you need to perform **DELETE** calls, so you don't need to set the authorization headers manually each time.

In [ ]:
def perform_delete(url):
    """
    Perform HTTP DEL with Authorization Headers
    :url: Specify the destination URL
    :return: Requests Response Object
    """
    return requests.delete(url=url, headers=headers)

# GET ALL PARTIES

API used to retrieve all the parties. I suggest to use the paginated API using the parameters:

    page: Number of the Page
    perPage: Number of Item per Page
    
Inside the Header will be returned the Link of the Next / Previous Page in the Key **Link**:

    <https://api.capsulecrm.com/api/v2/parties?page=2&perPage=10>; rel="next"

or

    <https://api.capsulecrm.com/api/v2/parties?page=1&perPage=10>; rel="prev"

In [ ]:
parties_url = 'https://api.capsulecrm.com/api/v2/parties'
parties_url_paginated = 'https://api.capsulecrm.com/api/v2/parties?page=2&perPage=10'

In [ ]:
resp = perform_get(parties_url)
parties = resp.json()
print parties.keys()
print len(parties['parties'])

In [ ]:
resp = perform_get(parties_url_paginated)

In [ ]:
print resp

In [ ]:
parties = resp.json()

In [ ]:
print parties.keys()
print len(parties['parties'])

In [ ]:
print resp.headers
print "\n--\n\nLink to next/prev Page:"
print resp.headers['Link']


# SEARCH PARTIES

Open Search API. We can search by **EMAIL**, if we have some results means that we have an user with that email.

In [ ]:
SEARCH_STRING = 'MAIL-HERE'

In [ ]:
search_url = 'https://api.capsulecrm.com/api/v2/parties/search?q=%s' % SEARCH_STRING

In [ ]:
resp = perform_get(search_url)

In [ ]:
print resp

In [ ]:
resp.json()

In [ ]:
parties = resp.json()
print parties['parties']

# Add New Party

Create new reference

In [ ]:
create_url = 'https://api.capsulecrm.com/api/v2/parties'

In [ ]:
data = {'party': {
            'about': 'Hello my name is Alex, i\'m just testing the API',
            'emailAddresses': [{'address': 'random@email.com'}],
            'type': 'person',
            'firstName': 'Alex',
            'lastName': 'Comu'
            }
       }
import json
data = json.dumps(data)
print data

In [ ]:
resp = perform_post(create_url, data)

In [ ]:
resp

In [ ]:
print resp.json()

In [ ]:
resp.headers

In [ ]:
user = resp.json()['party']
print user


# Update Party

Update reference given a specific ID.

In [ ]:
update_url = 'https://api.capsulecrm.com/api/v2/parties/{}'.format(user.get('id'))
print update_url

In [ ]:
data = {'party': {
            'type': 'person',
            'firstName': 'Alex',
            'lastName': 'Comunian'
            }
       }
import json
data = json.dumps(data)
print data

In [ ]:
resp = perform_put(update_url, data)

In [ ]:
print resp

In [ ]:
resp.json()

## PUT for not-existing resource

In [ ]:
update_url = 'https://api.capsulecrm.com/api/v2/parties/ashjdgajsdhjkasd'


In [ ]:
resp = perform_put(update_url, data)

In [ ]:
resp

In [ ]:
resp.json()

# Delete Party

In [ ]:
delete_url = 'https://api.capsulecrm.com/api/v2/parties/{}'.format(user.get('id'))
print delete_url

In [ ]:
resp = perform_delete(delete_url)

In [ ]:
print resp

In [ ]:
# if status is OK -> No Response
print resp.status_code

# Delete not existing Party

In [ ]:
delete_url = 'https://api.capsulecrm.com/api/v2/parties/asdasdasdasduastdguashjdbaksda'
print delete_url

In [ ]:
resp = perform_delete(delete_url)

In [ ]:
print resp

In [ ]:
resp.json()